In [1]:
import os
import statistics
import time

from dotenv import load_dotenv
from openai import OpenAI

# Загрузка переменных окружения
load_dotenv()

VLLM_MODEL_NAME = "Qwen/Qwen2.5-14B-Instruct"
VLLM_BASE_URL = os.getenv("VLLM_BASE_URL")
VLLM_API_KEY = os.getenv("VLLM_API_KEY")

# Настройка клиента vLLM
vllm_client = OpenAI(
    base_url=VLLM_BASE_URL, api_key=VLLM_API_KEY
)

display(VLLM_BASE_URL)

'http://154.20.254.95:50856/v1'

In [2]:
# Список тестовых запросов
test_queries = [
    "Какие основные правила безопасности на железной дороге?",
    "Расскажи о структуре РЖД",
    "Какие льготы доступны сотрудникам РЖД?",
    "Опиши процесс обучения новых сотрудников в РЖД",
    "Какие инновационные проекты реализует РЖД?"
]

# Функция для отправки запроса и измерения времени
def test_query(query):
    start_time = time.time()

    response = vllm_client.chat.completions.create(
        model=VLLM_MODEL_NAME,
        messages=[
            {"role": "system", "content": "Вы - ассистент РЖД, помогающий новым сотрудникам."},
            {"role": "user", "content": query}
        ],
        max_tokens=500  # Ограничиваем длину ответа для ускорения тестирования
    )

    end_time = time.time()
    elapsed_time = end_time - start_time

    return elapsed_time, response.choices[0].message.content

# Проведение тестов
results = []
for query in test_queries:
    print(f"Тестирование запроса: {query}")
    elapsed_time, response = test_query(query)
    results.append(elapsed_time)
    print(f"Время ответа: {elapsed_time:.2f} секунд")
    print(f"Ответ: {response}")  # Выводим первые 100 символов ответа
    print("-" * 50)

# Анализ результатов
avg_time = statistics.mean(results)
min_time = min(results)
max_time = max(results)

print(f"Среднее время ответа: {avg_time:.2f} секунд")
print(f"Минимальное время ответа: {min_time:.2f} секунд")
print(f"Максимальное время ответа: {max_time:.2f} секунд")


Тестирование запроса: Какие основные правила безопасности на железной дороге?


APIConnectionError: Connection error.

In [ ]:
from typing import Dict, List

from pydantic import BaseModel, ValidationError


# Определение Pydantic модели для структурированного вывода
class EmployeeBenefits(BaseModel):
    name: str
    position: str
    benefits: List[str]
    joining_date: str

# Имитация вызова модели Qwen-2.5 Instruct
def simulate_model_call(query: str) -> Dict:
    # В реальном сценарии здесь был бы вызов API модели
    time.sleep(1)  # Имитация задержки ответа
    return {
        'name': 'Иванов Иван',
        'position': 'Старший специалист',
        'benefits': ['Медицинская страховка', 'Транспортные льготы'],
        'joining_date': '2024-01-15',
        'reasoning': 'Сначала я определил, что запрос касается льгот сотрудника. '
                     'Затем я собрал информацию о типичных льготах в РЖД. '
                     'Наконец, я структурировал ответ согласно запрошенному формату.'
    }

# Тест на структурированный вывод и валидацию через Pydantic
def test_structured_output(query: str) -> Dict:
    response = simulate_model_call(query)
    try:
        validated_data = EmployeeBenefits(**response)
        print(f"Структурированный вывод успешно валидирован: {validated_data}")
        return response
    except ValidationError as e:
        print(f"Ошибка валидации: {e}")
        return {}

# Тест на Chain of Thought (CoT)
def test_chain_of_thought(query: str) -> bool:
    response = simulate_model_call(query)
    if 'reasoning' in response and len(response['reasoning'].split()) > 10:
        print(f"CoT присутствует: {response['reasoning']}")
        return True
    else:
        print("CoT отсутствует или недостаточно подробный")
        return False

# Чеклист для оценки качества ответа
def run_checklist(response: Dict) -> List[bool]:
    checklist = [
        lambda r: len(r['name']) > 0,
        lambda r: len(r['position']) > 0,
        lambda r: len(r['benefits']) > 0,
        lambda r: len(r['joining_date']) == 10,  # Формат даты YYYY-MM-DD
        lambda r: 'reasoning' in r and len(r['reasoning']) > 0
    ]
    return [check(response) for check in checklist]


query = "Какие льготы доступны сотрудникам РЖД?"

print("Тест 1: Структурированный вывод и Pydantic валидация")
structured_output = test_structured_output(query)

print("\nТест 2: Chain of Thought")
cot_present = test_chain_of_thought(query)

print("\nТест 3: Проверка по чеклисту")
checklist_results = run_checklist(structured_output)
print(f"Результаты чеклиста: {checklist_results}")

print("\nИтоговые результаты:")
print(f"Структурированный вывод: {'Успешно' if structured_output else 'Неуспешно'}")
print(f"Chain of Thought: {'Присутствует' if cot_present else 'Отсутствует'}")
print(f"Чеклист пройден: {all(checklist_results)}")